ss

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib
import re


In [3]:

# Load and preprocess data
df = pd.read_csv('PDFMalware2022.csv', header=0)

# Drop rows with missing 'Class' and invalid 'endobj' values
df = df.dropna(subset=['Class'])
df = df[df['endobj'] != 'pdfid.py']

# Replace NaN with 0 and -1 with 0
df = df.fillna(0).replace([-1, -1.00], 0)

# Convert categorical columns
df['Class'] = np.where(df['Class'].str.contains('Malicious'), 1, 0)
df['text'] = np.where(df['text'].str.contains('Yes'), 1, 0)

# Process header column
def header_obj(col):
    return 1 if re.search(r'%PDF-\d*\.?\d*', col) else 0

df['header'] = df['header'].apply(header_obj)

# Drop unnecessary columns
df = df.drop(['File name', 'JS', 'title characters'], axis=1)

# Prepare features and target
X = df.drop('Class', axis=1)
y = df['Class']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0, shuffle=True)

# Train Logistic Regression model
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)



['scaler.pkl']

saving

In [ ]:
# Save the model and scaler
joblib.dump(model, 'pdf_malware_model.pkl')
joblib.dump(scaler, 'scaler.pkl')